# Notebook 1: Préparation, Visualisation et Évaluation des Données

## Objectif
Ce notebook guide à travers les étapes de collecte de données brutes pour des paires et timeframes spécifiés, leur chargement, fusion, l'enrichissement avec des features techniques (y compris HMM et potentiellement CryptoBERT), la visualisation du dataset résultant, et sa sauvegarde pour les étapes suivantes (entraînement, backtesting).

## 1. Configuration Globale du Notebook

Modifiez les variables dans la cellule suivante pour configurer le notebook selon vos besoins (paires, dates, clés API, etc.).

In [ ]:
# --- Initialisation des Modules Python (Obligatoire en premier) ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os
import logging
import json
import subprocess
from pathlib import Path
import glob

# --- Paramètres de Configuration Principaux (À MODIFIER PAR L'UTILISATEUR) ---
TARGET_PAIRS_INPUT = ["BTC/USDT", "XRP/USDT", "BNB/USDT", "SHIB/USDT", "MATIC/USDT"]  # Liste ou string CSV
TARGET_TIMEFRAMES_INPUT = "1m" # String ou liste
START_DATE_INPUT = "2020-10-01"
END_DATE_INPUT = "2024-11-01"
EXCHANGE_ID_FOR_COLLECTION_INPUT = "binance"

# --- Booléens pour activer/désactiver des sections ---
RUN_DATA_COLLECTION = True
LOAD_FIXTURE_IF_EMPTY = True
RUN_FEATURE_ENGINEERING = True
RUN_DATA_VALIDATION = True

# --- Configuration des Clés API ---
API_KEYS_INPUT = {
    "BINANCE_API_KEY": "VOTRE_BINANCE_KEY_ICI_SI_NON_DEFINIE_EN_ENV",
    "BINANCE_API_SECRET": "VOTRE_BINANCE_SECRET_ICI_SI_NON_DEFINI_EN_ENV",
    "BITGET_API_KEY": "VOTRE_BITGET_KEY_ICI_SI_NON_DEFINIE_EN_ENV",
    "BITGET_API_SECRET": "VOTRE_BITGET_SECRET_ICI_SI_NON_DEFINI_EN_ENV",
    "BITGET_PASSPHRASE": "VOTRE_BITGET_PASSPHRASE_ICI_SI_NON_DEFINI_EN_ENV",
    "GOOGLE_API_KEY": "VOTRE_GOOGLE_KEY_ICI_SI_NON_DEFINIE_EN_ENV",
    "GOOGLE_CSE_ID": "VOTRE_GOOGLE_CSE_ID_ICI_SI_NON_DEFINI_EN_ENV",
    "GEMINI_API_KEY_1": "VOTRE_GEMINI_KEY_ICI_SI_NON_DEFINIE_EN_ENV",
    "OPENROUTER_API_KEY": "VOTRE_OPENROUTER_KEY_ICI_SI_NON_DEFINI_EN_ENV"
}

# --- Chemins ---
PROJECT_ROOT_NOTEBOOK_LEVEL = os.path.abspath(os.getcwd())
PROJECT_CODE_ROOT = os.path.join(PROJECT_ROOT_NOTEBOOK_LEVEL, "ultimate")
RAW_DATA_DIR_NAME = "data/raw/market"
PROCESSED_DATA_DIR_NAME = "data/processed"
PROCESSED_DATA_FILENAME = "multipaire.parquet"

# --- Initialisation (Logging, PYTHONPATH, Config, Variables Globales) ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
sns.set_theme(style="whitegrid"); plt.rcParams['figure.figsize'] = (18, 6); plt.rcParams['figure.dpi'] = 100

if PROJECT_CODE_ROOT not in sys.path: sys.path.append(PROJECT_CODE_ROOT); logger.info(f"'{PROJECT_CODE_ROOT}' ajouté au PYTHONPATH.")

cfg_instance = None; apply_feature_pipeline = None; load_and_split_data = None
try:
    from utils.feature_engineering import apply_feature_pipeline as apply_feature_pipeline_imported
    apply_feature_pipeline = apply_feature_pipeline_imported
    from model.training.data_loader import load_and_split_data as load_and_split_data_imported
    load_and_split_data = load_and_split_data_imported
    from config.config import Config
    logger.info("Modules projet importés.")
    cfg_instance = Config()
    logger.info(f"Instance de Config créée. Project root utilisé par Config: {getattr(cfg_instance, '_project_root', 'Non défini par la classe Config')}")
    if not cfg_instance.yaml_config:
        logger.error("CRITIQUE: config.yaml non chargé par Config(). Assurez-vous qu'il est dans 'Morningstar/config/config.yaml' et que la classe Config le trouve.")
except ImportError as e:
    logger.error(f"Erreur import modules projet: {e}. Certaines fonctionnalités seront désactivées.", exc_info=True)
    if cfg_instance is None:
        class FallbackConfig:
            _project_root = PROJECT_ROOT_NOTEBOOK_LEVEL
            yaml_config = {}
            def get_config(self, key, default=None): return default
        cfg_instance = FallbackConfig()
        logger.warning("Utilisation d'une instance Config de fallback suite à une erreur d'import.")
except Exception as e:
    logger.error(f"Erreur majeure lors de l'initialisation de Config: {e}. Utilisation de FallbackConfig.", exc_info=True)
    if cfg_instance is None:
        class FallbackConfig:
            _project_root = PROJECT_ROOT_NOTEBOOK_LEVEL
            yaml_config = {}
            def get_config(self, key, default=None): return default
        cfg_instance = FallbackConfig()
        logger.warning("Utilisation d'une instance Config de fallback suite à une erreur majeure.")

# --- Correction robuste pour TARGET_PAIRS et TARGET_TIMEFRAMES ---
def ensure_list(val):
    if isinstance(val, str):
        return [v.strip() for v in val.split(',') if v.strip()]
    elif isinstance(val, list):
        return [str(v).strip() for v in val if str(v).strip()]
    else:
        return []

TARGET_PAIRS = ensure_list(TARGET_PAIRS_INPUT)
TARGET_TIMEFRAMES = ensure_list(TARGET_TIMEFRAMES_INPUT)
START_DATE = START_DATE_INPUT
END_DATE = END_DATE_INPUT
EXCHANGE_ID_FOR_COLLECTION = EXCHANGE_ID_FOR_COLLECTION_INPUT

RAW_DATA_DIR = Path(PROJECT_CODE_ROOT) / RAW_DATA_DIR_NAME
PROCESSED_DATA_DIR = Path(PROJECT_CODE_ROOT) / PROCESSED_DATA_DIR_NAME
PROCESSED_DATA_OUTPUT_PATH = PROCESSED_DATA_DIR / PROCESSED_DATA_FILENAME
RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DATA_DIR.mkdir(parents=True, exist_ok=True)

for key, value in API_KEYS_INPUT.items():
    env_value = os.getenv(key)
    if "_A_COLLER_ICI_SI_NON_DEFINIE_EN_ENV" not in value and value.strip() != "":
        os.environ[key] = value; logger.info(f"Variable d'env '{key}' définie pour cette session.")
    elif not env_value:
        logger.warning(f"Clé API '{key}' non définie.")
    else:
        logger.info(f"Clé API '{key}' utilisée depuis l'environnement.")

logger.info(f"Configuration du notebook terminée. Paires: {TARGET_PAIRS}, Timeframes: {TARGET_TIMEFRAMES}, Période: {START_DATE}-{END_DATE}")

2025-05-20 11:56:55,123 - __main__ - INFO - Modules projet importés.
2025-05-20 11:56:55,124 - __main__ - INFO - Instance de Config créée. Project root utilisé par Config: /home/morningstar/Desktop/crypto_robot/Morningstar
2025-05-20 11:56:55,126 - __main__ - INFO - Variable d'env 'BINANCE_API_KEY' définie pour cette session.
2025-05-20 11:56:55,127 - __main__ - INFO - Variable d'env 'BINANCE_API_SECRET' définie pour cette session.
2025-05-20 11:56:55,129 - __main__ - INFO - Variable d'env 'BITGET_API_KEY' définie pour cette session.
2025-05-20 11:56:55,129 - __main__ - INFO - Variable d'env 'BITGET_API_SECRET' définie pour cette session.
2025-05-20 11:56:55,130 - __main__ - INFO - Variable d'env 'BITGET_PASSPHRASE' définie pour cette session.
2025-05-20 11:56:55,131 - __main__ - INFO - Variable d'env 'GOOGLE_API_KEY' définie pour cette session.
2025-05-20 11:56:55,132 - __main__ - INFO - Variable d'env 'GOOGLE_CSE_ID' définie pour cette session.
2025-05-20 11:56:55,133 - __main__ - IN

## 2. Collecte des Données Brutes (Optionnel)

Cette section exécute le script `ultimate/data_collectors/market_data_collector.py` si `RUN_DATA_COLLECTION` est `True`.

In [7]:
if RUN_DATA_COLLECTION:
    logger.info("Début de la collecte de données brutes via market_data_collector.py...")
    collector_script_path = os.path.join(PROJECT_CODE_ROOT, "data_collectors", "market_data_collector.py")
    
    if not os.path.exists(collector_script_path):
        logger.error(f"Script de collecte non trouvé: {collector_script_path}. Vérifiez le chemin.")
    elif not TARGET_PAIRS or not TARGET_TIMEFRAMES:
        logger.error("TARGET_PAIRS ou TARGET_TIMEFRAMES non définis. Collecte annulée.")
    else:
        pairs_str = ",".join(TARGET_PAIRS)
        timeframes_str = ",".join(TARGET_TIMEFRAMES)
        command = [
            sys.executable, collector_script_path,
            "--exchange", EXCHANGE_ID_FOR_COLLECTION,
            "--pairs", pairs_str, "--timeframes", timeframes_str,
            "--start-date", START_DATE, "--end-date", END_DATE,
            "--output-dir", str(RAW_DATA_DIR)
        ]
        logger.info(f"Exécution de la commande de collecte: {' '.join(command)}")
        try:
            process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, cwd=PROJECT_CODE_ROOT)
            stdout, stderr = process.communicate(timeout=1800) # Timeout de 30 minutes
            
            logger.info("--- Sortie Standard du Script de Collecte ---")
            for line in stdout.splitlines(): logger.info(f"[COLLECTOR] {line}")
            logger.info("--- Fin Sortie Standard ---")
            
            if process.returncode == 0:
                logger.info("Collecte de données terminée avec succès.")
            else:
                logger.error(f"Le script de collecte de données a échoué avec le code de retour {process.returncode}.")
            if stderr:
                logger.error("--- Erreurs du Script de Collecte ---")
                for line in stderr.splitlines(): logger.error(f"[COLLECTOR_ERR] {line}")
                logger.error("--- Fin Erreurs Collecte ---")
        except subprocess.TimeoutExpired:
            logger.error("La collecte de données a dépassé le délai de 30 minutes.", exc_info=True)
            if 'process' in locals() and process.poll() is None: process.kill()
        except Exception as e:
            logger.error(f"Erreur lors de l'exécution du script de collecte: {e}", exc_info=True)
else:
    logger.info("Collecte de données brutes sautée (RUN_DATA_COLLECTION=False). Le notebook tentera de charger des fichiers existants.")

2025-05-20 11:56:55,156 - __main__ - INFO - Collecte de données brutes sautée (RUN_DATA_COLLECTION=False). Le notebook tentera de charger des fichiers existants.


## 3. Chargement et Fusion des Données Brutes

Charge les fichiers Parquet individuels (un par paire/timeframe) depuis `ultimate/data/raw/market/` et les fusionne.

In [8]:
all_loaded_dfs = []
df_raw_combined = pd.DataFrame() 
logger.info(f"Chargement des fichiers Parquet depuis {RAW_DATA_DIR} pour {TARGET_PAIRS} et {TARGET_TIMEFRAMES}...")

if TARGET_PAIRS and TARGET_TIMEFRAMES and isinstance(TARGET_PAIRS, list) and isinstance(TARGET_TIMEFRAMES, list):
    for pair in TARGET_PAIRS:
        for tf in TARGET_TIMEFRAMES:
            safe_pair_name = pair.replace('/', '')
            expected_filename = f"{safe_pair_name}_{tf}.parquet"
            file_path = RAW_DATA_DIR / expected_filename
            if file_path.exists():
                logger.info(f"Chargement de {file_path}...")
                try:
                    df_temp = pd.read_parquet(file_path)
                    df_temp['pair'] = pair
                    df_temp['timeframe'] = tf
                    if 'timestamp' in df_temp.columns:
                        df_temp['timestamp'] = pd.to_datetime(df_temp['timestamp'], errors='coerce', utc=True)
                    elif isinstance(df_temp.index, pd.DatetimeIndex):
                         df_temp = df_temp.reset_index().rename(columns={'index':'timestamp'})
                         df_temp['timestamp'] = pd.to_datetime(df_temp['timestamp'], errors='coerce', utc=True)
                    else:
                        logger.error(f"Colonne 'timestamp' ou index Datetime manquant dans {file_path}. Fichier ignoré.")
                        continue 
                    df_temp.dropna(subset=['timestamp'], inplace=True) 
                    if not df_temp.empty:
                        all_loaded_dfs.append(df_temp)
                        logger.info(f"Chargé {file_path}. Shape: {df_temp.shape}. Colonnes: {df_temp.columns.tolist()[:5]}...")
                    else:
                        logger.warning(f"DataFrame vide après traitement de timestamp pour {file_path}.")
                except Exception as e:
                    logger.error(f"Erreur chargement {file_path}: {e}", exc_info=True)
            else:
                logger.warning(f"Fichier non trouvé: {file_path}.")
else:
    logger.warning("TARGET_PAIRS ou TARGET_TIMEFRAMES non définis correctement.")

if all_loaded_dfs:
    try:
        df_raw_combined = pd.concat(all_loaded_dfs, ignore_index=True)
        if 'timestamp' in df_raw_combined.columns and not df_raw_combined.empty:
            df_raw_combined = df_raw_combined.sort_values(by=['pair', 'timeframe', 'timestamp']).reset_index(drop=True)
            logger.info(f"Données brutes fusionnées et triées. Shape finale: {df_raw_combined.shape}")
            print("\n--- Aperçu des Données Brutes Combinées (df_raw_combined) ---")
            print(df_raw_combined.head())
            print("\n--- Statistiques par Paire/Timeframe ---")
            print(df_raw_combined.groupby(['pair', 'timeframe']).size().reset_index(name='counts'))
            if not df_raw_combined.empty:
                logger.info("\n--- Graphiques des Prix de Clôture (Bruts) par Paire/Timeframe ---")
                for (pair_val, tf_val), group in df_raw_combined.groupby(['pair', 'timeframe']):
                    plt.figure(figsize=(18,4))
                    group.set_index('timestamp')['close'].plot(title=f'{pair_val} - {tf_val} - Prix de Clôture (Brut)')
                    plt.ylabel('Prix de Clôture'); plt.xlabel('Timestamp'); plt.grid(True); plt.tight_layout()
                    plt.show()
        elif df_raw_combined.empty:
            logger.warning("df_raw_combined est vide après tentative de fusion.")
        else: 
            logger.error("Colonne 'timestamp' manquante après fusion. df_raw_combined invalidé.")
            df_raw_combined = pd.DataFrame()
    except Exception as e:
        logger.error(f"Erreur concaténation/tri: {e}", exc_info=True)
        df_raw_combined = pd.DataFrame()
else:
    logger.warning("Aucune donnée brute chargée pour les cibles.")

if df_raw_combined.empty and LOAD_FIXTURE_IF_EMPTY:
    logger.warning("Utilisation du fichier de fixture car aucune donnée cible n'a été chargée ou la fusion a échoué.")
    fixture_path = os.path.join(PROJECT_CODE_ROOT, "tests", "fixtures", "golden_backtest.parquet")
    if os.path.exists(fixture_path):
        try:
            df_raw_combined = pd.read_parquet(fixture_path)
            if 'pair' not in df_raw_combined.columns: df_raw_combined['pair'] = (TARGET_PAIRS[0] if (TARGET_PAIRS and isinstance(TARGET_PAIRS, list)) else 'BTC/USDT')
            if 'timeframe' not in df_raw_combined.columns: df_raw_combined['timeframe'] = (TARGET_TIMEFRAMES[0] if (TARGET_TIMEFRAMES and isinstance(TARGET_TIMEFRAMES, list)) else '1h')
            if 'timestamp' in df_raw_combined.columns: df_raw_combined['timestamp'] = pd.to_datetime(df_raw_combined['timestamp'], utc=True)
            logger.info(f"Données de fixture chargées. Shape: {df_raw_combined.shape}")
            print(df_raw_combined.head())
        except Exception as e:
            logger.error(f"Erreur chargement fixture {fixture_path}: {e}")
            df_raw_combined = pd.DataFrame()
    else:
        logger.error(f"Fichier de fixture {fixture_path} non trouvé. Le DataFrame restera vide.")
elif df_raw_combined.empty and not LOAD_FIXTURE_IF_EMPTY:
    logger.error("Aucune donnée chargée et chargement de fixture désactivé. Le notebook ne peut pas continuer sans données.")

2025-05-20 11:56:55,192 - __main__ - INFO - Chargement des fichiers Parquet depuis /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/data/raw/market pour ['BTC/USDT', 'XRP/USDT', 'BNB/USDT', 'SHIB/USDT', 'MATIC/USDT'] et ['1m']...
2025-05-20 11:56:55,197 - __main__ - WARNING - Fichier non trouvé: /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/data/raw/market/BTCUSDT_1m.parquet.
2025-05-20 11:56:55,198 - __main__ - WARNING - Fichier non trouvé: /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/data/raw/market/XRPUSDT_1m.parquet.
2025-05-20 11:56:55,200 - __main__ - WARNING - Fichier non trouvé: /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/data/raw/market/BNBUSDT_1m.parquet.
2025-05-20 11:56:55,200 - __main__ - WARNING - Fichier non trouvé: /home/morningstar/Desktop/crypto_robot/Morningstar/ultimate/data/raw/market/SHIBUSDT_1m.parquet.
2025-05-20 11:56:55,201 - __main__ - WARNING - Fichier non trouvé: /home/morningstar/Desktop/crypto_robot/

## 4. Enrichissement & Calcul des Features

Applique `apply_feature_pipeline` (de `ultimate/utils/feature_engineering.py`) sur le DataFrame combiné.
Cette fonction est responsable de calculer tous les indicateurs techniques, les features HMM, et potentiellement les features LLM (CryptoBERT).

In [9]:
df_feat = pd.DataFrame() 
if RUN_FEATURE_ENGINEERING:
    if df_raw_combined is not None and not df_raw_combined.empty:
        logger.info("Début de l'enrichissement des données et du calcul des features...")
        try:
            if 'apply_feature_pipeline' in globals() and callable(apply_feature_pipeline):
                if 'pair' in df_raw_combined.columns and 'timeframe' in df_raw_combined.columns and df_raw_combined[['pair', 'timeframe']].nunique().prod() > 1:
                    logger.info("Application du pipeline de features par groupe (paire, timeframe)...")
                    df_feat_list = []
                    for name_tuple, group_df in df_raw_combined.groupby(['pair', 'timeframe']):
                        logger.info(f"Traitement des features pour {name_tuple} (Shape: {group_df.shape})...")
                        group_for_feat = group_df.copy()
                        if 'timestamp' in group_for_feat.columns:
                            group_for_feat = group_for_feat.set_index('timestamp') 
                        elif not isinstance(group_for_feat.index, pd.DatetimeIndex):
                            logger.error(f"Groupe {name_tuple} sans index timestamp valide. Features non calculées.")
                            continue
                        processed_group = apply_feature_pipeline(group_for_feat) 
                        if processed_group is not None and not processed_group.empty:
                            processed_group['pair'] = name_tuple[0]
                            processed_group['timeframe'] = name_tuple[1]
                            df_feat_list.append(processed_group.reset_index()) 
                        else:
                            logger.warning(f"apply_feature_pipeline a retourné None/vide pour {name_tuple}.")
                    if df_feat_list:
                        df_feat = pd.concat(df_feat_list, ignore_index=True)
                        if 'timestamp' in df_feat.columns: 
                            df_feat = df_feat.sort_values(by=['pair', 'timeframe', 'timestamp']).reset_index(drop=True)
                else: 
                    logger.info("Application du pipeline de features sur le DataFrame entier...")
                    df_temp_for_feat = df_raw_combined.copy()
                    if 'timestamp' in df_temp_for_feat.columns:
                         df_temp_for_feat = df_temp_for_feat.set_index('timestamp')
                    elif not isinstance(df_temp_for_feat.index, pd.DatetimeIndex):
                        logger.error("DataFrame n'a pas d'index timestamp valide. Features non calculées.")
                        df_temp_for_feat = None
                    if df_temp_for_feat is not None:
                         df_feat = apply_feature_pipeline(df_temp_for_feat)
                         if df_feat is not None: 
                             df_feat = df_feat.reset_index() 
                             if 'pair' not in df_feat.columns and 'pair' in df_raw_combined.columns: df_feat['pair'] = df_raw_combined['pair'].iloc[0] if len(df_raw_combined['pair'].unique()) == 1 else 'MULTI_PAIR'
                             if 'timeframe' not in df_feat.columns and 'timeframe' in df_raw_combined.columns: df_feat['timeframe'] = df_raw_combined['timeframe'].iloc[0] if len(df_raw_combined['timeframe'].unique()) == 1 else 'MULTI_TF'
            else:
                logger.error("La fonction `apply_feature_pipeline` n'a pas été importée ou n'est pas callable. Enrichissement sauté.")
            
            if df_feat is not None and not df_feat.empty:
                logger.info(f"Dataset enrichi. Shape finale: {df_feat.shape}")
                print("\n--- Aperçu du Dataset Enrichi (df_feat) ---")
                print(df_feat.head())
                logger.info(f"Colonnes de df_feat: {df_feat.columns.tolist()}")
            else:
                logger.error("L'application du pipeline de features a résulté en un DataFrame vide ou None.")
                df_feat = pd.DataFrame() 
        except Exception as e:
            logger.error(f"Erreur lors de apply_feature_pipeline: {e}", exc_info=True)
            df_feat = pd.DataFrame()
    else:
        logger.warning("Enrichissement sauté car df_raw_combined est vide ou non chargé.")
else:
    logger.info("Calcul des features désactivé (RUN_FEATURE_ENGINEERING=False).")

2025-05-20 11:56:55,242 - __main__ - WARNING - Enrichissement sauté car df_raw_combined est vide ou non chargé.


## 5. Vérification Détaillée des Features et Visualisations Post-Enrichissement

Cette section analyse le DataFrame `df_feat` après l'application du pipeline de features.

In [10]:
if RUN_DATA_VALIDATION and df_feat is not None and not df_feat.empty:
    logger.info("\n--- Vérification Détaillée et Visualisation des Features ---")
    print(f"Shape de df_feat: {df_feat.shape}")
    print(f"Nombre total de colonnes dans df_feat: {len(df_feat.columns)}")
    print(f"Colonnes présentes: {df_feat.columns.tolist()}")

    # Vérification des NaNs
    nan_counts = df_feat.isnull().sum()
    nan_percent = (nan_counts / len(df_feat)) * 100
    nan_summary = pd.DataFrame({'NaN_Count': nan_counts, 'NaN_Percent': nan_percent})
    print("\n--- Résumé des Valeurs Manquantes (NaN) par Colonne ---")
    print(nan_summary[nan_summary['NaN_Count'] > 0].sort_values(by='NaN_Percent', ascending=False))

    # Vérification CryptoBERT
    bert_cols = [col for col in df_feat.columns if col.startswith('bert_')]
    if bert_cols:
        logger.info(f"{len(bert_cols)} features CryptoBERT trouvées.")
        nan_bert_percent = df_feat[bert_cols].isnull().sum().sum() / (df_feat[bert_cols].size if df_feat[bert_cols].size > 0 else 1) * 100
        logger.info(f"Pourcentage de NaN dans features CryptoBERT: {nan_bert_percent:.2f}%")
        if nan_bert_percent > 50:
            logger.warning("Beaucoup de NaNs dans les features CryptoBERT. Vérifiez la source de données textuelles.")
    else:
        logger.warning("Aucune feature CryptoBERT ('bert_*') trouvée.")

    # Vérification HMM
    hmm_regime_col = 'hmm_regime'
    hmm_prob_cols = [col for col in df_feat.columns if col.startswith('hmm_prob_')]
    if hmm_regime_col in df_feat.columns:
        logger.info(f"Feature HMM '{hmm_regime_col}' trouvée. Distribution:")
        if 'pair' in df_feat.columns and 'timeframe' in df_feat.columns:
            try:
                print(df_feat.groupby(['pair', 'timeframe'])[hmm_regime_col].value_counts(normalize=True, dropna=False).unstack(fill_value=0).round(3))
                if df_feat[hmm_regime_col].nunique() > 1: 
                    g = sns.catplot(data=df_feat, x=hmm_regime_col, col='pair', row='timeframe', kind='count', sharey=False, height=3.5, aspect=1.2, palette='viridis')
                    g.fig.suptitle('Distribution des Régimes HMM par Paire/Timeframe', y=1.03); plt.tight_layout(); plt.show()
                else:
                    logger.info(f"Une seule valeur unique pour {hmm_regime_col}, graphique countplot non affiché.")
            except Exception as e:
                logger.warning(f"Erreur visualisation régimes HMM: {e}")
        else:
            print(df_feat[hmm_regime_col].value_counts(normalize=True, dropna=False))
    else:
        logger.warning(f"Feature HMM '{hmm_regime_col}' non trouvée. C'est une colonne de label importante.")
    if hmm_prob_cols:
        logger.info(f"{len(hmm_prob_cols)} features de probabilité HMM trouvées.")
        nan_hmm_prob_percent = df_feat[hmm_prob_cols].isnull().sum().sum() / (df_feat[hmm_prob_cols].size if df_feat[hmm_prob_cols].size > 0 else 1) * 100
        logger.info(f"Pourcentage de NaN dans probabilités HMM: {nan_hmm_prob_percent:.2f}%")
    else:
        logger.warning("Aucune feature de probabilité HMM ('hmm_prob_*') trouvée.")

    # Visualisation de features techniques clés
    key_tech_features = [col for col in ['close', 'feature_SMA_10', 'feature_RSI_14', 'MACD', 'BBM', 'ATR', hmm_regime_col] if col in df_feat.columns]
    if len(key_tech_features) > 1 and 'timestamp' in df_feat.columns and 'pair' in df_feat.columns and 'timeframe' in df_feat.columns:
        logger.info(f"Visualisation de features techniques clés: {key_tech_features}")
        for (pair_val, tf_val), group in df_feat.groupby(['pair', 'timeframe']):
            plot_df = group.set_index('timestamp')
            actual_cols_to_plot = [col for col in key_tech_features if col in plot_df.columns]
            if len(actual_cols_to_plot) > 0:
                plot_df[actual_cols_to_plot].plot(subplots=True, figsize=(18, 2.5*len(actual_cols_to_plot)), layout=(-1,1), sharex=True, title=f'Features Clés pour {pair_val} - {tf_val}')
                plt.tight_layout(); plt.show()
            else:
                logger.warning(f"Aucune des features {key_tech_features} à plotter pour {pair_val} - {tf_val}")

        # Matrice de corrélation
        if df_feat['pair'].nunique() > 0 and df_feat['timeframe'].nunique() > 0:
            grouped_for_corr = df_feat.groupby(['pair', 'timeframe'])
            if grouped_for_corr.groups:
                first_group_key = list(grouped_for_corr.groups.keys())[0]
                first_group_df = grouped_for_corr.get_group(first_group_key)
                cols_for_corr = [col for col in ['close', 'volume', 'feature_RSI_14', 'MACD', 'ATR', hmm_regime_col] if col in first_group_df.columns and first_group_df[col].dtype in [np.float64, np.int64]]
                if len(cols_for_corr) > 1:
                    plt.figure(figsize=(10, 8))
                    correlation_matrix = first_group_df[cols_for_corr].corr()
                    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
                    plt.title(f'Matrice de Corrélation pour {first_group_key}')
                    plt.show()
            else:
                logger.warning("Aucun groupe trouvé pour la matrice de corrélation.")
    else:
        logger.warning(f"Pas assez de features/colonnes pour visualisation détaillée.")
elif RUN_DATA_VALIDATION:
    logger.warning("Vérification détaillée et visualisation des features sautées car df_feat est vide.")
else:
    logger.info("Vérification détaillée et visualisation des features désactivées (RUN_DATA_VALIDATION=False).")

2025-05-20 11:56:55,290 - __main__ - WARNING - Vérification détaillée et visualisation des features sautées car df_feat est vide.


## 6. Export du Dataset Enrichi

Sauvegarde de `df_feat` dans `PROCESSED_DATA_OUTPUT_PATH`.

In [11]:
# --- Génération automatique de 'market_regime' si manquante ---
if df_feat is not None and not df_feat.empty:
    if 'market_regime' not in df_feat.columns:
        logger.warning("La colonne 'market_regime' est manquante — génération automatique à partir de la variation de 'close'.")
        df_feat['market_regime'] = (df_feat['close'].diff() > 0).astype(int).fillna(0).astype(int)
        logger.info(f"'market_regime' générée, distribution :\n{df_feat['market_regime'].value_counts(dropna=False)}")

    # --- Sauvegarde du dataset enrichi ---
    try:
        df_feat.to_parquet(PROCESSED_DATA_OUTPUT_PATH, index=False)
        logger.info(f"Dataset enrichi sauvegardé dans {PROCESSED_DATA_OUTPUT_PATH}")
        # Relecture pour vérification
        df_check = pd.read_parquet(PROCESSED_DATA_OUTPUT_PATH)
        exists = 'market_regime' in df_check.columns
        logger.info(f"'market_regime' présente après relecture ? {exists}")
    except Exception as e:
        logger.error(f"Erreur lors de l’export / vérification : {e}", exc_info=True)
        raise
else:
    logger.warning("Export sauté car df_feat est vide ou non chargé.")

2025-05-20 11:56:55,322 - __main__ - WARNING - Export sauté car df_feat est vide ou non chargé.


## 7. Vérification de la Compatibilité pour l'Entraînement

Utilise `load_and_split_data` pour s'assurer que le dataset sauvegardé peut être chargé et contient les colonnes de labels attendues.

In [12]:
# --- Cellule 6 : Vérification et sauvegarde du dataset enrichi ---
if df_feat is not None and not df_feat.empty:
    # 1. Génération automatique de 'market_regime' si manquante
    if 'market_regime' not in df_feat.columns:
        logger.warning("La colonne 'market_regime' est manquante — génération automatique à partir de la variation de 'close'.")
        # Simple rule-based regime: 1 si la clôture monte par rapport à la précédente, sinon 0
        df_feat['market_regime'] = (df_feat['close'].diff() > 0).astype(int).fillna(0).astype(int)
        logger.info(f"'market_regime' générée, distribution :\n{df_feat['market_regime'].value_counts(dropna=False)}")

    # 2. Sauvegarde
    try:
        df_feat.to_parquet(PROCESSED_DATA_OUTPUT_PATH, index=False)
        logger.info(f"Dataset enrichi sauvegardé dans {PROCESSED_DATA_OUTPUT_PATH}")
        # relecture pour vérification
        df_check = pd.read_parquet(PROCESSED_DATA_OUTPUT_PATH)
        exists = 'market_regime' in df_check.columns
        logger.info(f"'market_regime' présente après relecture ? {exists}")
    except Exception as e:
        logger.error(f"Erreur lors de l’export / vérification : {e}", exc_info=True)
        raise
else:
    logger.warning("Export sauté car df_feat est vide ou non chargé.")


2025-05-20 11:56:55,344 - __main__ - WARNING - Export sauté car df_feat est vide ou non chargé.
